In [1]:
import brightway2 as bw
import pandas as pd
import numpy as np

Using environment variable BRIGHTWAY2_DIR for data directory:
/data/user/kim_a/Brightway3


In [2]:
bw.projects.set_current('GSA for oases')
co = bw.Database('CH consumption 1.0')
average_consump = co.search('ch hh average consumption')[0]
# average_consump = co.search('food')[0]

In [3]:
sectors = [act for act in co if 'sector' in act["name"]]

In [4]:
set([act for act in co if act.get("category_coarse") is None])

{'Alcoholic beverages and tobacco sector' (1 month of consumption, CH, None),
 'Clothing and footwear sector' (1 month of consumption, CH, None),
 'Communication sector' (1 month of consumption, CH, None),
 'Durable goods sector' (1 month of consumption, CH, None),
 'Education sector' (1 month of consumption, CH, None),
 'Fees sector' (1 month of consumption, CH, None),
 'Food and non-alcoholic beverages sector' (1 month of consumption, CH, None),
 'Furnishings, household equipment and routine household maintenance sector' (1 month of consumption, CH, None),
 'Health sector' (1 month of consumption, CH, None),
 'Housing, water, electricity, gas and other fuels sector' (1 month of consumption, CH, None),
 'Miscellaneous goods and services sector' (1 month of consumption, CH, None),
 'Other insurance premiums sector' (1 month of consumption, CH, None),
 'Premiums for life insurance sector' (1 month of consumption, CH, None),
 'Recreation and culture sector' (1 month of consumption, CH, N

In [5]:
# list(average_consump.exchanges())

In [16]:
list_functional_units = []
for exc in average_consump.exchanges():
    if exc['type'] != 'production':
        list_functional_units.append({exc['input']: exc['amount']})

# list_functional_units = []
# for act in sectors:
#     list_functional_units.append({act: 1})
        
list_methods = [('IPCC 2013', 'climate change', 'GTP 100a')]

In [17]:
bw.calculation_setups['av_consump_contribution_analysis'] = {'inv':list_functional_units, 'ia':list_methods}

In [18]:
%%time
myMultiLCA = bw.MultiLCA('av_consump_contribution_analysis')

CPU times: user 1min 3s, sys: 5.7 s, total: 1min 8s
Wall time: 3.75 s


In [19]:
lcia_unit = bw.Method(list_methods[0]).metadata['unit']

In [20]:
fus = [bw.get_activity(list(el.keys())[0])['name'][:25] for el in list_functional_units]
df = pd.DataFrame(index=fus, columns=[lcia_unit], data=myMultiLCA.results)
# df['exchanges'] = [list(fu.values())[0] for fu in list_functional_units]
df['units'] = [bw.get_activity(list(el.keys())[0])['unit'] for el in list_functional_units]


In [21]:
df.sort_values(lcia_unit, ascending=False, inplace=True)

In [22]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)

                            kg CO2-Eq                 units
Natural gas and other fue  295.471929             megajoule
Gasoline                   292.822565                 litre
Desktop computers           84.940072                  unit
Diesel                      74.600249                 litre
Electricity of principal    41.602612         kilowatt hour
Portable computers          32.526744                  unit
Refuse collection costs a   18.883158  number of waste bags
Hard and semi-hard cheese   15.893403              kilogram
Sausages, cold meat and p   13.549018              kilogram
Printers (incl. multifunc   11.976351                  unit
Chocolate                   11.220741              kilogram
Beef                        10.039435              kilogram
Fresh, soft and melted ch    9.440092              kilogram
Whole milk                   8.846425                 litre
Pork, fresh or frozen        8.670563              kilogram
Pure and ground coffee       7.162286   

In [ ]:
df["kg CO2-Eq"].values.sum()

In [27]:
# df_ = df.sort_values(lcia_unit)
# df_['kg CO2-Eq'].plot.barh(figsize=(20,40))

# Biosphere

In [ ]:
import bw2analyzer

In [ ]:
lca = bw.LCA({average_consump: 1}, list_methods[0])

In [ ]:
lca.lci()
lca.lcia()
lca.score

In [ ]:
ca = bw2analyzer.ContributionAnalysis()

In [ ]:
ca.annotated_top_processes(lca)

# Contribution analysis for sectors

In [ ]:
list_methods = [('IPCC 2013', 'climate change', 'GTP 100a')]
list_fus = []
for cat_of_interest in category_activities:
    list_fus.append({co.search(cat_of_interest)[0]: 1})
    
bw.calculation_setups['sector_contribution_analysis'] = {'inv':list_fus, 'ia':list_methods}

%%time
myMultiLCA = bw.MultiLCA('sector_contribution_analysis')

lcia_unit = bw.Method(list_methods[0]).metadata['unit']

fus = [bw.get_activity(list(el.keys())[0])['name'][:25] for el in list_fus]
df = pd.DataFrame(index=fus, columns=[lcia_unit], data=myMultiLCA.results)
df['units'] = [bw.get_activity(list(el.keys())[0])['unit'] for el in list_fus]

df.sort_values(lcia_unit, ascending=False, inplace=True)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)